# GPT

가장 주인공 같은 인물, 혹은 가장 움직이는게 좋을 거 같은 인물 하나를 지정해서 설명해달라고 해야할 것 같다.

seg_prompt는 위치로 인물을 지정하는 게 가장 정확한 듯하다. 그리고 손에 들고 있는 것까지 언급을 해주면 다 같이 마스킹 할 수 있게끔 하였다. 근데 다른 공통적인 물건까지 포함시키면 전혀 다른 위치의 해당 물건이 같은 마스크가 되어버린다.

motion_prompt는 다른 물건 언급 없이 동작 자체만 설명하는 것이 좋을 것 같다. 

In [ ]:
# gpt 프롬프트 입니다.

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are an expert in analyzing storybook illustrations and describing animations for human-like characters. Return results in a dictionary format."},
        {
            "role": "user", "content": """
Analyze the provided storybook page image.
1. Regocnize the texts and illustrations on the storybook image.
2. Choose one human character that you think would be best to animate.
3. Describe the character's position on the image and what he/she is holding.
4. Suggest a simple motion in HumanML3D style based on context. You should focus only on the character's motion and do not include any other objects.

Output Format:
{   [
        "name": "One-word name for the character",
        "description": "Name and location and object. ex) A boy on left side holding a gun.",
        "motion": "Simple motion description for the character in HumanML3D dataset style. Start with 'A person is'."
    ]
}
"""
        },
        {"role": "user", "content": [{"type": "image_url", "image_url": {"url":  f"data:image/jpeg;base64,{base64_image}"}}]}
    ]
)

# 여기부터 돌리세요

In [1]:
import os
import json

image_name = "a10"
data_dir= "/home/jovyan/data/axe/" + image_name + "/"
image_path = data_dir + image_name + ".png"
char_dir = data_dir + "char/"
os.makedirs(char_dir, exist_ok=True)
inpaint_dir = data_dir + "inpaint/"
os.makedirs(inpaint_dir, exist_ok=True)

motion_path = char_dir + image_name + ".bvh"

seg_prompt = "A girl wearing blue dress."
motion_prompt = "\"\""

env_vars = {
    "data_dir": data_dir,
    "image_name": image_name,
    "image_path": image_path,
    "char_dir": char_dir,
    "motion_path": motion_path,
    "inpaint_dir": inpaint_dir,
    "seg_prompt": seg_prompt,
    "motion_prompt": motion_prompt
}

with open("env_vars.json", "w") as f:
    json.dump(env_vars, f)

# Segmentation

In [1]:
import json

# 파일에서 환경변수 읽기
with open("env_vars.json", "r") as f:
    env_vars = json.load(f)

# 불러온 환경변수 사용
data_dir = env_vars.get("data_dir")
image_name = env_vars.get("image_name")
image_path = env_vars.get("image_path")
char_dir = env_vars.get("char_dir")
motion_path = env_vars.get("motion_path")
inpaint_dir = env_vars.get("inpaint_dir")
seg_prompt = env_vars.get("seg_prompt")
motion_prompt = env_vars.get("motion_prompt")

In [2]:
from PIL import Image
from lang_sam import LangSAM
import numpy as np
import cv2
import yaml

/home/jovyan/miniconda3/envs/langsam/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = LangSAM(sam_type="sam2.1_hiera_large")

In [ ]:
image_pil = Image.open(image_path).convert("RGB")
result = model.predict([image_pil], [seg_prompt])[0]

mask = np.logical_or.reduce(result["masks"]).astype(int)
mask = np.asarray(mask * 255, dtype=np.uint8)

kernel = np.ones((10, 10), np.uint8)
dilated_mask = cv2.dilate(mask, kernel, iterations=1)

bbox = np.array([np.min(result["boxes"][:, 0]), np.min(result["boxes"][:, 1]), np.max(result["boxes"][:, 2]), np.max(result["boxes"][:, 3])], dtype=np.float32)
l, t, r, b = [round(x) for x in bbox]
cropped_image = image_pil.crop(bbox)
cropped_mask = mask[t:b, l:r]

with open(char_dir + "/bounding_box.yaml", 'w') as f:
        yaml.dump({
            'left': l,
            'top': t,
            'right': r,
            'bottom': b
        }, f)

cropped_image.save(char_dir + "/texture.png")
mask = Image.fromarray(mask)
dilated_mask = Image.fromarray(dilated_mask)
cropped_mask = Image.fromarray(cropped_mask)

mask.save(data_dir + image_name + "_mask.png")
dilated_mask.save(inpaint_dir + image_name + "_mask.png")
cropped_mask.save(char_dir + "mask.png")

Predicting 1 masks
Predicted 1 masks


# Inpainting

In [1]:
import json

# 파일에서 환경변수 읽기
with open("env_vars.json", "r") as f:
    env_vars = json.load(f)

# 불러온 환경변수 사용
data_dir = env_vars.get("data_dir")
image_name = env_vars.get("image_name")
image_path = env_vars.get("image_path")
char_dir = env_vars.get("char_dir")
motion_path = env_vars.get("motion_path")
inpaint_dir = env_vars.get("inpaint_dir")
seg_prompt = env_vars.get("seg_prompt")
motion_prompt = env_vars.get("motion_prompt")

In [2]:
import shutil
shutil.copy(image_path, inpaint_dir + image_name + ".png")

!cd lama && export TORCH_HOME=$(pwd) && export PYTHONPATH=$(pwd) && python bin/predict.py model.path=$(pwd)/big-lama indir={inpaint_dir} outdir={inpaint_dir}

Detectron v2 is not installed
[2024-12-05 16:22:43,457][saicinpainting.utils][WARNING] - Setting signal 10 handler <function print_traceback_handler at 0x7f1c0e73c2f0>
[2024-12-05 16:22:43,490][root][INFO] - Make training model default
[2024-12-05 16:22:43,490][saicinpainting.training.trainers.base][INFO] - BaseInpaintingTrainingModule init called
[2024-12-05 16:22:43,490][root][INFO] - Make generator ffc_resnet
[2024-12-05 16:22:43,912][saicinpainting.training.trainers.base][INFO] - Generator
FFCResNetGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): FFC_BN_ACT(
      (ffc): FFC(
        (convl2l): Conv2d(4, 64, kernel_size=(7, 7), stride=(1, 1), bias=False, padding_mode=reflect)
        (convl2g): Identity()
        (convg2l): Identity()
        (convg2g): Identity()
        (gate): Identity()
      )
      (bn_l): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn_g): Identity()
      (act_l): ReLU(inplace=True)


# Pose Estimation

In [1]:
import json

# 파일에서 환경변수 읽기
with open("env_vars.json", "r") as f:
    env_vars = json.load(f)

# 불러온 환경변수 사용
data_dir = env_vars.get("data_dir")
image_name = env_vars.get("image_name")
image_path = env_vars.get("image_path")
char_dir = env_vars.get("char_dir")
motion_path = env_vars.get("motion_path")
inpaint_dir = env_vars.get("inpaint_dir")
seg_prompt = env_vars.get("seg_prompt")
motion_prompt = env_vars.get("motion_prompt")

In [2]:
import torch, torchvision
import mmpose
from IPython.display import Image, display
import cv2
import os
import yaml
import numpy as np
from mmpose.apis import (inference_top_down_pose_model, init_pose_model, vis_pose_result)

/home/jovyan/miniconda3/envs/pose/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jovyan/miniconda3/envs/pose/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [3]:
pose_config = '/home/jovyan/mmpose/config.py'
pose_checkpoint = '/home/jovyan/mmpose/best_AP_epoch_72.pth'
cropped = cv2.imread(char_dir + "/texture.png")

pose_model = init_pose_model(pose_config, pose_checkpoint)

pose_results, _ = inference_top_down_pose_model(
    pose_model, char_dir + "/texture.png", person_results=None)

# vis_result = vis_pose_result(
#     pose_model,
#     char_dir + "/image.png",
#     pose_results)

# cv2.imwrite("test.png", vis_result)
# display(Image("test.png"))

load checkpoint from local path: /home/jovyan/mmpose/best_AP_epoch_72.pth


In [4]:
kpts = np.array(pose_results[0]['keypoints'])[:, :2]

skeleton = []
skeleton.append({'loc' : [round(x) for x in (kpts[11]+kpts[12])/2], 'name': 'root'          , 'parent': None})
skeleton.append({'loc' : [round(x) for x in (kpts[11]+kpts[12])/2], 'name': 'hip'           , 'parent': 'root'})
skeleton.append({'loc' : [round(x) for x in (kpts[5]+kpts[6])/2  ], 'name': 'torso'         , 'parent': 'hip'})
skeleton.append({'loc' : [round(x) for x in  kpts[0]             ], 'name': 'neck'          , 'parent': 'torso'})
skeleton.append({'loc' : [round(x) for x in  kpts[6]             ], 'name': 'right_shoulder', 'parent': 'torso'})
skeleton.append({'loc' : [round(x) for x in  kpts[8]             ], 'name': 'right_elbow'   , 'parent': 'right_shoulder'})
skeleton.append({'loc' : [round(x) for x in  kpts[10]            ], 'name': 'right_hand'    , 'parent': 'right_elbow'})
skeleton.append({'loc' : [round(x) for x in  kpts[5]             ], 'name': 'left_shoulder' , 'parent': 'torso'})
skeleton.append({'loc' : [round(x) for x in  kpts[7]             ], 'name': 'left_elbow'    , 'parent': 'left_shoulder'})
skeleton.append({'loc' : [round(x) for x in  kpts[9]             ], 'name': 'left_hand'     , 'parent': 'left_elbow'})
skeleton.append({'loc' : [round(x) for x in  kpts[12]            ], 'name': 'right_hip'     , 'parent': 'root'})
skeleton.append({'loc' : [round(x) for x in  kpts[14]            ], 'name': 'right_knee'    , 'parent': 'right_hip'})
skeleton.append({'loc' : [round(x) for x in  kpts[16]            ], 'name': 'right_foot'    , 'parent': 'right_knee'})
skeleton.append({'loc' : [round(x) for x in  kpts[11]            ], 'name': 'left_hip'      , 'parent': 'root'})
skeleton.append({'loc' : [round(x) for x in  kpts[13]            ], 'name': 'left_knee'     , 'parent': 'left_hip'})
skeleton.append({'loc' : [round(x) for x in  kpts[15]            ], 'name': 'left_foot'     , 'parent': 'left_knee'})


char_cfg = {'skeleton': skeleton, 'height': cropped.shape[0], 'width': cropped.shape[1]}

with open(char_dir + '/char_cfg.yaml', 'w') as f:
        yaml.dump(char_cfg, f)

joint_overlay = cropped.copy()
for joint in skeleton:
        x, y = joint['loc']
        name = joint['name']
        cv2.circle(joint_overlay, (int(x), int(y)), 5, (0, 0, 0), 5)
        cv2.putText(joint_overlay, name, (int(x), int(y+15)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, 2)
cv2.imwrite(char_dir + '/joint_overlay.png', joint_overlay)

True

# Motion Generation

In [1]:
import json

# 파일에서 환경변수 읽기
with open("env_vars.json", "r") as f:
    env_vars = json.load(f)

# 불러온 환경변수 사용
data_dir = env_vars.get("data_dir")
image_name = env_vars.get("image_name")
image_path = env_vars.get("image_path")
char_dir = env_vars.get("char_dir")
motion_path = env_vars.get("motion_path")
inpaint_dir = env_vars.get("inpaint_dir")
seg_prompt = env_vars.get("seg_prompt")
motion_prompt = env_vars.get("motion_prompt")

In [ ]:
!cd momask-codes && python gen_t2m.py --gpu_id 0 --ext exp1 --motion_length 100 --text_prompt ""{motion_prompt}""

------------ Options -------------
batch_size: 32
checkpoints_dir: ./checkpoints
cond_scale: 4
dataset_name: t2m
dropout: 0.2
ext: exp1
ff_size: 1024
force_mask: False
gpu_id: 0
gumbel_sample: False
is_train: False
latent_dim: 384
mask_edit_section: None
max_motion_length: 196
motion_length: 100
n_heads: 6
n_layers: 8
name: t2m_nlayer8_nhead6_ld384_ff1024_cdp0.1_rvq6ns
num_batch: 2
repeat_times: 1
res_name: tres_nlayer8_ld384_ff1024_rvq6ns_cdp0.2_sw
seed: 10107
source_motion: example_data/000612.npy
temperature: 1.0
text_path: 
text_prompt: A person raises right arm straight to the right.
time_steps: 18
topkr: 0.9
unit_length: 4
use_res_model: False
vq_name: rvq_nq1_dc512_nc512
which_epoch: latest
-------------- End ----------------
Reading ./checkpoints/t2m/t2m_nlayer8_nhead6_ld384_ff1024_cdp0.1_rvq6ns/opt.txt
Reading ./checkpoints/t2m/rvq_nq6_dc512_nc512_noshare_qdp0.2/opt.txt
Loading VQ Model rvq_nq6_dc512_nc512_noshare_qdp0.2 Completed!
Reading ./checkpoints/t2m/tres_nlayer8_ld384_

In [ ]:
import shutil
import glob
import os

source_dir = '/home/jovyan/momask-codes/generation/exp1/animations/0'
files = glob.glob(f'{source_dir}/*.bvh')
latest_file = max(files, key=os.path.getctime)
shutil.copy(latest_file, motion_path)

files = glob.glob(f'{source_dir}/*.mp4')
latest_file = max(files, key=os.path.getctime)
shutil.copy(latest_file, char_dir+image_name+".mp4")

'/home/jovyan/data/perse/p8/char/p8.mp4'

# Animating

In [1]:
import json

# 파일에서 환경변수 읽기
with open("env_vars.json", "r") as f:
    env_vars = json.load(f)

# 불러온 환경변수 사용
data_dir = env_vars.get("data_dir")
image_name = env_vars.get("image_name")
image_path = env_vars.get("image_path")
char_dir = env_vars.get("char_dir")
motion_path = env_vars.get("motion_path")
inpaint_dir = env_vars.get("inpaint_dir")
seg_prompt = env_vars.get("seg_prompt")
motion_prompt = env_vars.get("motion_prompt")

In [2]:
import yaml
from PIL import Image

image = char_dir + "texture.png"  # 이미지 파일 경로
with Image.open(image) as img:
    width, height = img.size

window_width = int(width * 1.5)
window_height = int(height * 1.5)

yaml_path = "/home/jovyan/data/animating/config/mvc/pbl.yaml"

new_character_cfg = char_dir + "char_cfg.yaml"
video_path = data_dir+ image_name + ".gif"

with open(yaml_path, 'r') as file:
    config = yaml.safe_load(file)

config['scene']['ANIMATED_CHARACTERS'][0]['character_cfg'] = new_character_cfg
config['controller']['OUTPUT_VIDEO_PATH'] = video_path
config['view']['WINDOW_DIMENSIONS'] = [window_width, window_height]

with open(yaml_path, 'w') as file:
    yaml.dump(config, file, default_flow_style=False)


yaml_path = "/home/jovyan/data/animating/config/motion/test.yaml"

new_bvh = motion_path

with open(yaml_path, 'r') as file:
    config = yaml.safe_load(file)

config['filepath'] = motion_path

with open(yaml_path, 'w') as file:
    yaml.dump(config, file, default_flow_style=False)

In [7]:
from animated_drawings import render
render.start('/home/jovyan/data/animating/config/mvc/pbl.yaml')

point [[0.11174786 0.260745  ]] not inside or on edge of any triangle in mesh. Skipping it
point [[0.07449856 0.19627507]] not inside or on edge of any triangle in mesh. Skipping it
point [[0.04011461 0.1661891 ]] not inside or on edge of any triangle in mesh. Skipping it
point [[0.67191976 0.14756447]] not inside or on edge of any triangle in mesh. Skipping it
point [[0.6418338  0.09455588]] not inside or on edge of any triangle in mesh. Skipping it
point [[0.68481374 0.06160459]] not inside or on edge of any triangle in mesh. Skipping it
 Writing video to: /home/jovyan/data/axe/a10/a10.gif


100%|██████████| 100/100 [00:08<00:00, 11.78it/s]


In [8]:
from moviepy.editor import VideoFileClip, ImageClip, CompositeVideoClip
import yaml

yaml_file = char_dir + "bounding_box.yaml"  # YAML 파일 경로
with open(yaml_file, 'r') as file:
    bounding_box = yaml.safe_load(file)

# Bounding Box 정보 추출
left = bounding_box['left']
right = bounding_box['right']
top = bounding_box['top']
bottom = bounding_box['bottom']

# 2. Bounding Box 중심 계산
center_x = (left + right) / 2
center_y = (top + bottom) / 2

# 1. GIF 파일 로드
video_path = data_dir+ image_name + ".gif"
gif_file = video_path  # GIF 파일 경로

clip = VideoFileClip(gif_file, has_mask=True)

adjusted_x = center_x - (clip.w / 2)  # 중심 기준 좌표
adjusted_y = center_y - (clip.h / 2)

# 2. 배경 준비 (이미지 파일 또는 색상으로 배경 생성)
background_file = inpaint_dir + image_name + "_mask.png"  # 배경 이미지 경로
background = ImageClip(background_file)  # 배경 이미지 로드

background = background.set_duration(clip.duration).set_fps(clip.fps)

# 3. 배경과 GIF 결합
# CompositeVideoClip을 사용하여 배경과 GIF를 겹칩니다.
final_clip = CompositeVideoClip([
    background,
    clip.set_position((adjusted_x, adjusted_y))  # 중심을 맞춘 좌표로 이동
])

# 4. 동영상으로 저장
final_clip.write_videofile(image_name + ".mp4")

Moviepy - Building video a10.mp4.
Moviepy - Writing video a10.mp4



Moviepy - Done !
Moviepy - video ready a10.mp4
